In [103]:
import pandas as pd
import numpy as np

In [104]:
df = pd.read_csv("spotify_millsongdata.csv")


In [105]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [106]:
df.shape

(57650, 4)

In [107]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [108]:
df=df.drop('link' , axis = 1).reset_index(drop = True)

In [109]:
df['text'][0]

"Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [110]:
df = df.sample(5000)

In [111]:
df =df.reset_index(drop = True)

In [112]:
df.shape

(5000, 3)

Text Pre-processing

In [113]:
df['text'] = df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n', ' ',regex = True)

In [114]:
import nltk 
from nltk.stem.porter import PorterStemmer

In [115]:
stemmy = PorterStemmer()

In [116]:
import nltk
from nltk.stem import PorterStemmer

def token(txt):
    # Tokenize the input text
    tokens = nltk.word_tokenize(txt)
    
    # Create a PorterStemmer object
    stemmer = PorterStemmer()
    
    # Apply stemming to each token
    stemmed_tokens = [stemmer.stem(w) for w in tokens]
    
    # Join the stemmed tokens into a string
    result = " ".join(stemmed_tokens)
    
    return result



In [117]:
token(' you are beautiful,beauty')

'you are beauti , beauti'

In [118]:
df['text'].apply ( lambda x:  token( x ))

0       la la hold insid of her the horribl dream of b...
1       i 've been watch you for day now babi i just l...
2       do you think about the moon ? made you feel re...
3       who 'll be the next in line ? who 'll be the n...
4       pick me up , put me down push me in , turn me ...
                              ...                        
4995    young dro , young dro ladi and gentlemen , thi...
4996    thi man kept wishin that the devil would come ...
4997    oh happi day ( oh happi day ) oh happi day ( o...
4998    he would swear by hi mouth almighti i wa live ...
4999    i came to you and never ask too much wonder wh...
Name: text, Length: 5000, dtype: object

In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [120]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [121]:
matrix = tfidf.fit_transform(df['text'])

In [122]:
matrix

<5000x23667 sparse matrix of type '<class 'numpy.float64'>'
	with 268741 stored elements in Compressed Sparse Row format>

In [123]:
similar =  cosine_similarity(matrix)

In [124]:
similar[0]

array([1.        , 0.01428388, 0.08585677, ..., 0.22763606, 0.00972629,
       0.0228336 ])

In [125]:
df[df['song']=='Never Gonna Leave You']

,artist,song,text


In [126]:
def recommendation(song_name):
    idx = df[df['song'] == song_name].index[0]
    distances = sorted(list(enumerate(similar[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for s_id in distances[1:21]:
        songs.append(df.iloc[s_id[0]].song)
        
    return songs

In [129]:
recommendation('Wrap It Up')


['Ultra Violet',
 'Baby Hold On',
 'Baby Baby',
 "I'm Gonna Cry (Cry Baby)",
 'Oh My Soul',
 'Baby',
 "I Don't Know Why",
 'Pack It Up',
 "Since You've Been Gone",
 'Baby Can Dance',
 'The Only One',
 'Shining Star',
 'Rain In My Heart',
 "I've Got To Give It All To You",
 'Superstar',
 'I Want Elvis For Christmas',
 'Ghost Rider',
 'Roll With My Baby',
 'In The Mood',
 'Still In Love']

In [128]:
df

,artist,song,text
0,Xiu Xiu,Dr. Troll,la la holds inside of her \r the horrible dre...
1,Eurythmics,Wrap It Up,i've been watching you for days now baby \r i...
2,Idina Menzel,Think Too Much,do you think about the moon? \r made you feel...
3,Queens Of The Stone Age,Who'll Be The Next In Line,who'll be the next in line? \r who'll be the ...
4,Genesis,In Hiding,"pick me up, put me down \r push me in, turn m..."
...,...,...,...
4995,Young Dro,What It Is,"young dro, young dro \r \r ladies and gentl..."
4996,Halloween,Old Andy Kay Keeter Snapoose,this man kept wishin that the devil would come...
4997,Kirk Franklin,Oh Happy Day,oh happy day (oh happy day) \r oh happy day (...
4998,Wet Wet Wet,Wishing I Was Lucky,he would swear by his mouth almighty \r \r ...


In [130]:
import pickle

In [135]:
pickle.dump(similar , open("similarity","wb"))

In [136]:

pickle.dump(df , open("df","wb"))